In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook


with open('/Users/sofiawang/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [2]:
!pip install tqdm

In [3]:
with open('/Users/sofiawang/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)

yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [4]:
search_results = yelp_api.search_query(location='SF, CA',
                                       term='Thai')
print(type(search_results))
search_results.keys()

<class 'dict'>


dict_keys(['businesses', 'total', 'region'])

In [5]:
LOCATION = 'SF,CA'
TERM = 'Thai'

In [6]:
JSON_FILE = "Data/results_in_progress_SF_thai.json"
JSON_FILE

'Data/results_in_progress_SF_thai.json'

In [7]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_SF_thai.json already exists.


In [8]:
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [9]:
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [10]:
total_results= results['total']
total_results

1100

In [11]:
import pandas as pd

biz = pd.DataFrame(search_results['businesses'])
biz.head(2) 

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,8kck3-K4zYKTJbJko0JlXQ,farmhouse-kitchen-thai-cuisine-san-francisco,Farmhouse Kitchen Thai Cuisine,https://s3-media4.fl.yelpcdn.com/bphoto/kUlEaP...,False,https://www.yelp.com/biz/farmhouse-kitchen-tha...,3692,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...",4.0,"{'latitude': 37.760192883932795, 'longitude': ...","[pickup, delivery, restaurant_reservation]",$$,"{'address1': '710 Florida St', 'address2': '',...",+14158142920,(415) 814-2920,2201.588556
1,tZmiZw49SNRVzF6h1Jfx9g,ben-thai-cafe-san-francisco,Ben Thai Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/UVXRkn...,False,https://www.yelp.com/biz/ben-thai-cafe-san-fra...,980,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...",4.5,"{'latitude': 37.78924, 'longitude': -122.42065}","[pickup, delivery, restaurant_reservation]",$$,"{'address1': '1331 Polk St', 'address2': None,...",+14157712562,(415) 771-2562,3445.440156


In [12]:
results_per_page = len(results['businesses'])
results_per_page

20

In [13]:
import time, math

n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

55

In [14]:
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [15]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [17]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/55 [00:00<?, ?it/s]

In [19]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

In [20]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[!] Data/results_in_progress_SF_thai.json already exists. Deleting previous file...
[i] Data/results_in_progress_SF_thai.json not found. Saving empty list to new file.
- 0 previous results found.


55

In [21]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/55 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [22]:
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,8kck3-K4zYKTJbJko0JlXQ,farmhouse-kitchen-thai-cuisine-san-francisco,Farmhouse Kitchen Thai Cuisine,https://s3-media4.fl.yelpcdn.com/bphoto/kUlEaP...,False,https://www.yelp.com/biz/farmhouse-kitchen-tha...,3692,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...",4.0,"{'latitude': 37.760192883932795, 'longitude': ...","[delivery, pickup, restaurant_reservation]",$$,"{'address1': '710 Florida St', 'address2': '',...",+14158142920,(415) 814-2920,2201.588556
1,tZmiZw49SNRVzF6h1Jfx9g,ben-thai-cafe-san-francisco,Ben Thai Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/UVXRkn...,False,https://www.yelp.com/biz/ben-thai-cafe-san-fra...,980,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...",4.5,"{'latitude': 37.78924, 'longitude': -122.42065}","[delivery, pickup, restaurant_reservation]",$$,"{'address1': '1331 Polk St', 'address2': None,...",+14157712562,(415) 771-2562,3445.440156
2,AzFqspVE3gUWYaEselXLUA,khob-khun-thai-cuisine-and-breakfast-san-franc...,Khob Khun Thai Cuisine & Breakfast,https://s3-media3.fl.yelpcdn.com/bphoto/vmbbOH...,False,https://www.yelp.com/biz/khob-khun-thai-cuisin...,226,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.5,"{'latitude': 37.78107129493808, 'longitude': -...","[delivery, pickup]",$$,"{'address1': '3741 Geary Blvd', 'address2': No...",+14153794382,(415) 379-4382,3032.307778
3,s2R3RUJ09eBFvHVIUS4Shg,kothai-republic-san-francisco,Kothai Republic,https://s3-media3.fl.yelpcdn.com/bphoto/eWlrib...,False,https://www.yelp.com/biz/kothai-republic-san-f...,234,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.5,"{'latitude': 37.762389, 'longitude': -122.466065}","[delivery, pickup]",NaN,"{'address1': '1398 9th Ave', 'address2': None,...",+14157424058,(415) 742-4058,2606.596929
4,8LFfuBSFK9YfHp1r-jsJEw,oraan-thai-eatery-san-francisco,Oraan Thai Eatery,https://s3-media2.fl.yelpcdn.com/bphoto/8buUkX...,False,https://www.yelp.com/biz/oraan-thai-eatery-san...,27,"[{'alias': 'thai', 'title': 'Thai'}]",4.5,"{'latitude': 37.78141, 'longitude': -122.459837}",[],NaN,"{'address1': '3750 Geary Blvd', 'address2': ''...",+16282197598,(628) 219-7598,3074.144773


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,JPIsHSqd1-6tefmKNEc_Tg,xing-yuan-chinese-restaurant-alameda,Xing Yuan Chinese Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/3CPTiT...,False,https://www.yelp.com/biz/xing-yuan-chinese-res...,311,"[{'alias': 'chinese', 'title': 'Chinese'}]",3.5,"{'latitude': 37.783673065657, 'longitude': -12...","[delivery, pickup]",$$,"{'address1': '839 Marina Village Pkwy', 'addre...",+15105228215,(510) 522-8215,14571.753546
996,ouAw6oihgt3Ricwcbf1fMA,pho-anh-dao-alameda-alameda,Pho Anh Dao Alameda,https://s3-media2.fl.yelpcdn.com/bphoto/pk5SKY...,False,https://www.yelp.com/biz/pho-anh-dao-alameda-a...,160,"[{'alias': 'vietnamese', 'title': 'Vietnamese'}]",3.5,"{'latitude': 37.778843, 'longitude': -122.276938}",[delivery],$$,"{'address1': '1919 Webster St', 'address2': ''...",+15102638577,(510) 263-8577,14161.625676
997,fcdjnsgO8Z5LthXUx3y-lA,pho-k-and-k-berkeley,Pho K & K,https://s3-media2.fl.yelpcdn.com/bphoto/_Xn2_u...,False,https://www.yelp.com/biz/pho-k-and-k-berkeley?...,432,"[{'alias': 'vietnamese', 'title': 'Vietnamese'}]",3.5,"{'latitude': 37.86448, 'longitude': -122.25822}","[delivery, pickup]",$,"{'address1': '2533 Telegraph Ave', 'address2':...",+15106669966,(510) 666-9966,19424.770032
998,cG8i_Ttq_MVGd7Ja5MTysQ,katsumi-sushi-berkeley,Katsumi Sushi,https://s3-media3.fl.yelpcdn.com/bphoto/vLJhmc...,False,https://www.yelp.com/biz/katsumi-sushi-berkele...,95,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",3.5,"{'latitude': 37.86821, 'longitude': -122.25803}","[delivery, pickup]",$,"{'address1': '2521 Durant Ave', 'address2': No...",+15105294799,(510) 529-4799,19698.176084
999,CNlCi4xU4FsVp0METnrT5g,chef-hungs-san-francisco,Chef Hung's,https://s3-media2.fl.yelpcdn.com/bphoto/DqiJfg...,False,https://www.yelp.com/biz/chef-hungs-san-franci...,184,"[{'alias': 'chinese', 'title': 'Chinese'}]",3.5,"{'latitude': 37.79405, 'longitude': -122.40665}",[delivery],$,"{'address1': '823 Clay St', 'address2': '', 'a...",+14153986883,(415) 398-6883,4526.095051


In [24]:
final_df.duplicated(subset='id').sum()

0

In [25]:
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0

In [26]:
final_df.to_csv('Data/final_results_NY_pizza.csv.gz', compression='gzip',index=False)